# 📊 감성 분석 + 숙소 추천 대시보드 (Streamlit)

In [ ]:

# Streamlit 설치
# !pip install streamlit pandas sentence-transformers scikit-learn


In [ ]:

import streamlit as st
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# 1. 데이터 불러오기
@st.cache_data
def load_data():
    df = pd.read_csv("숙소_리뷰_감성_분석_결과.csv")
    df = df[['name', 'review_content', 'sentiment_tfidf']].dropna()
    df = df[df['sentiment_tfidf'].isin(['긍정', '부정', '중립'])].reset_index(drop=True)
    return df.head(500)

df = load_data()
st.title("📍 전주 숙소 리뷰 감성 분석 대시보드")

# 2. 질문 입력
st.subheader("🔎 원하는 숙소를 자연어로 검색해보세요")
query = st.text_input("예: 청결하고 조용한 곳 알려줘", "")

# 3. 리뷰 임베딩
model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1')
review_embeddings = model.encode(df['review_content'], convert_to_tensor=True)

if query:
    query_embedding = model.encode([query])
    sim_scores = cosine_similarity(query_embedding, review_embeddings)[0]
    df['유사도'] = sim_scores
    top_results = df.sort_values(by='유사도', ascending=False).head(5)

    st.subheader("🏨 유사한 숙소 리뷰 추천")
    for i, row in top_results.iterrows():
        st.markdown(f"**숙소명**: {row['name']}  
**리뷰**: {row['review_content']}  
**감성**: {row['sentiment_tfidf']}  
**유사도**: {row['유사도']:.3f}")
        st.markdown("---")
